函数装饰器用于在源码中“标记”函数，以某种方式增强函数的行为。这是一项强大的功能，但是需先理解闭包。除了在装饰器中有用处之外，闭包还是回调式异步编程和函数式编程风格的基础。

## 7.1 装饰器的基础知识

装饰器是可调用的对象，其参数是另一个函数（被装饰的函数）。装饰器可能会处理被装饰的函数，然后把它返回，或者将其替换成另一个函数或可调用对象。

例1 装饰器通常把函数替换成另一个函数：

In [1]:
def deco(func):
    def inner():
        print("running inner()")
    return inner

In [2]:
@deco
def target():
    print("running target()")

In [3]:
target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner()>

严格来说，装饰器只是语法糖。装饰器可以像常规的可调用对象那样调用，其参数是另一个函数。在元编程（在运行时改变程序的行为）时，经常被运用。

综上，装饰器的一大特性是，能把被装饰的函数替换成其他函数。第二大特性是，装饰器在加载模块时立即执行。

## 7.2 Python何时执行装饰器

装饰器的一大关键特性是，它们在被装饰的函数定义之后立即运行。这通常在导入时（即Python 加载模块时），即运行了装饰器函数。

In [7]:
registry = []

def register(func):
    pass

## 7.4 变量作用域规则

例2 一个函数，读取一个局部变量和一个全局变量

In [8]:
def f1(a):
    print(a)
    print(b)

In [10]:
f1(3)

3


NameError: name 'b' is not defined

In [11]:
b = 6
f1(2)

2
6


例3 b是局部变量，因为在函数的定义体中给他赋值了：

In [13]:
def f2(a):
    print(a)
    print(b)
    b = 9

In [14]:
b = 6
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

注意，首先输出了 3，这表明 print(a) 语句执行了。但是第二个语句 print(b) 执行不了。一开始我很吃惊，我觉得会打印 6，因为有个全 局变量 b，而且是在 print(b) 之后为局部变量 b 赋值的。

可事实是，Python 编译函数的定义体时，它判断 b 是局部变量，因为在 函数中给它赋值了。生成的字节码证实了这种判断，Python 会尝试从本 地环境获取 b。后面调用 f2(3) 时， f2 的定义体会获取并打印局部变 量 a 的值，但是尝试获取局部变量 b 的值时，发现 b 没有绑定值。

** 这不是缺陷，而是设计选择：Python 不要求声明变量，但是假定在函数定义体中赋值的变量是局部变量。**

如果在函数中赋值时想让解释器把 b 当成全局变量，要使用 **global** 关键字声 明：

In [15]:
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [16]:
b = 6
f3(3)

3
6


In [17]:
b

9

In [18]:
f3(1)

1
9


In [19]:
b = 30
f3(20)

20
30


In [20]:
from dis import dis

In [21]:
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [22]:
dis(f2)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  4          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


其实上述是提取f1, f2函数运行的字节码，没怎么弄懂这是啥，留个疑问在这吧！！！？？？

## 7.5 闭包

很多人会把闭包和匿名函数弄混，这是因为在函数内部定义函数是不常见的，直到开始使用匿名函数才会这么做。而且，只有涉及嵌套函数时才有闭包问题。

其实，闭包指的是延伸了作用域的函数，其中包含函数定义体中引用，但是不在定义体中定义的非全局变量。函数是不是匿名的没有关系，关键是它能访问定义体之外定义的非全局变量。

假如有个名为 avg 的函数，它的作用是计算不断增加的系列值的均值； 例如，整个历史中某个商品的平均收盘价。每天都会增加新价格，因此 平均值要考虑至目前为止所有的价格。

* 使用类实现

In [62]:
class Averager:
    def __init__(self):
        self.series = []
        
    def __call__(self, value):
        self.series.append(value)
        total = sum(self.series)
        return total / len(self.series)

In [63]:
avg = Averager()

In [64]:
avg.cal_avg(10)

10.0

In [65]:
avg.cal_avg(11)

10.5

In [26]:
avg(10)

10.0

In [27]:
avg(11)

10.5

In [28]:
avg(12)

11.0

* 使用函数式实现，使用高阶函数`make_averager`

In [29]:
def make_averager():
    series = []
    
    def averager(value):
        series.append(value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [30]:
avg = make_averager()

In [31]:
avg(10)

10.0

In [32]:
avg(11)

10.5

In [33]:
avg(12)

11.0

In [35]:
avg.__code__.co_varnames

('value', 'total')

In [36]:
avg.__code__.co_freevars

('series',)

In [38]:
avg(10)

10.75

In [39]:
avg2 = make_averager()

In [40]:
series = [1, 2, 3]

In [41]:
avg2(10)

10.0

In [42]:
avg2(11)

10.5

综上，闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定。

注意，只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量。

## 7.6 nonlocal声明

前面实现 make_averager 函数的方法效率不高。在上述例子中，我们 把所有值存储在历史数列中，然后在每次调用 averager 时使用 sum 求 和。更好的实现方式是，只存储目前的总值和元素个数，然后使用这两 个数计算均值。

例：计算移动平均值的高阶函数，不保存所有历史值，但有 缺陷：

In [43]:
def make_averager():
    count = 0
    total = 0
    
    def averager(value):
        count += 1
        total += value
        
        return total / count
    
    return averager

In [44]:
avg = make_averager()

In [45]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

问题是，当 count 是数字或任何不可变类型时，count += 1 语句的作 用其实与 count = count + 1 一样。因此，我们在 averager 的定义 体中为 count 赋值了，这会把 count 变成局部变量。total 变量也受 这个问题影响。

上节示例没遇到这个问题，因为我们没有给 series 赋值，我们只是调用 series.append，并把它传给 sum 和 len。也就是说，我们利用了列表是可变的对象这一事实。

但是对数字、字符串、元组等不可变类型来说，只能读取，不能更新。如果尝试重新绑定，例如 count = count + 1，其实会隐式创建局部变量 count。这样，count 就不是自由变量了，因此不会保存在闭包中。

** 为了解决这个问题，Python 3 引入了 nonlocal 声明。它的作用是把变 量标记为自由变量，即使在函数中为变量赋予新值了，也会变成自由变 量。如果为 nonlocal 声明的变量赋予新值，闭包中保存的绑定会更 新。最新版 make_averager 的正确实现如下例所示。**

例： 计算移动平均值，不保存所有历史（使用 nonlocal 修 正）：

In [54]:
def make_averager():
    count = 0
    total = 0
    
    def averager(value):
        nonlocal count, total
        count += 1
        total += value
        
        return total / count
    
    return averager

In [55]:
avg = make_averager()

In [56]:
avg(10)

10.0

## 7.7 实现一个简单的装饰

In [57]:
def f(a):
    print(a)
    print(b)

In [58]:
f(3)

3
9
